In [ ]:
!pip uninstall -y msspackages

In [ ]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

In [ ]:
from msspackages import setup_runner
setup_runner()

In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
awswrangler_pandas_dataframe_to_s3(pod_training_data,  "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [ ]:
write_tensor(pod_training_tensor, "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [2]:
time_steps = 25
batch_size = 10
train_n_samples = batch_size * 1000
test_n_samples = round((batch_size*1000*0.2) / 0.8)

In [3]:
features = ['container_cpu_utilization','container_memory_utilization','container_memory_working_set']
scaled_features = ['scaled_container_cpu_utilization','scaled_container_memory_utilization','scaled_container_memory_working_set']

In [4]:
import boto3
import io
import pandas as pd

# Read the parquet file
buffer = io.BytesIO()
s3 = boto3.resource('s3')
object = s3.Object('dish-5g.core.pd.g.dp.eks.logs.e','container_pca_ad/v0.0.1/data/pandas_df/raw_training_2022_5_5.parquet')
object.download_fileobj(buffer)
container_training_df = pd.read_parquet(buffer)

print(container_training_df.head())

                Timestamp                              container_name_pod_id  \
0 2022-05-05 16:07:19.221   kube-multus-a40a4405-e9d9-48cc-8dc8-8f8175705d2d   
1 2022-05-05 16:07:19.806  csi-provisioner-b7e4f538-694e-40cb-ba1b-07ee8f...   
2 2022-05-05 16:07:20.954   aws-ip-mgmt-9bab717c-15eb-4a60-9c5a-448eb923dd7e   
3 2022-05-05 16:07:21.073          ifxr-3646e92e-c32e-4cab-a056-0f9c394308ed   
4 2022-05-05 16:07:21.832  zts-vaultfs-sidecar-4a55a3fe-7de3-4373-aea5-c4...   

   container_cpu_utilization  container_memory_utilization  \
0                   0.000000                      0.017143   
1                   0.001519                      0.008002   
2                   0.024498                      0.030183   
3                   0.010086                      0.080542   
4                   0.016788                      0.038478   

   container_memory_working_set  
0                      45838336  
1                      21397504  
2                      80703488  
3         

In [5]:
container_training_df.shape

(53645105, 5)

In [6]:
container_training_df['freq'] = container_training_df.groupby('container_name_pod_id')['container_name_pod_id'].transform('count')

In [7]:
filtered_container_df = container_training_df[container_training_df["freq"] > 25]

In [8]:
filtered_container_df.head()

,Timestamp,container_name_pod_id,container_cpu_utilization,container_memory_utilization,container_memory_working_set,freq
0,2022-05-05 16:07:19.221,kube-multus-a40a4405-e9d9-48cc-8dc8-8f8175705d2d,0.000000,0.017143,45838336,1176
1,2022-05-05 16:07:19.806,csi-provisioner-b7e4f538-694e-40cb-ba1b-07ee8f...,0.001519,0.008002,21397504,1138
2,2022-05-05 16:07:20.954,aws-ip-mgmt-9bab717c-15eb-4a60-9c5a-448eb923dd7e,0.024498,0.030183,80703488,1148
3,2022-05-05 16:07:21.073,ifxr-3646e92e-c32e-4cab-a056-0f9c394308ed,0.010086,0.080542,215359488,1153
4,2022-05-05 16:07:21.832,zts-vaultfs-sidecar-4a55a3fe-7de3-4373-aea5-c4...,0.016788,0.038478,102883328,1147


In [9]:
selected_container_list = filtered_container_df['container_name_pod_id'].sample(10000).to_list()

In [10]:
len(selected_container_list)

10000

In [11]:
selected_container_list

['day0-eks-bootstrap-e2201407-2b35-49f3-9f19-29a7e6b09053',
 'csi-driver-registrar-b02b80a0-55d6-4bc3-bb0e-ee4f6f76710b',
 'envoy-sidecar-fa236162-e1ae-42d8-a640-7d12a9b85fdc',
 'registrationcontainer-5aac7f6a-767f-40eb-b25c-42e34b04396c',
 'liveness-probe-7a4f3362-1b63-4600-ac77-5637bd4a54ed',
 'liveness-probe-b86713b8-197b-4245-a76e-1877a840391c',
 'envoy-sidecar-14fd1406-401c-4018-9421-051ba28fd52f',
 'kube-multus-d2246c42-0b1a-49d8-a4e3-41c5a76b08e8',
 'dsreader-container-098240d1-21d8-4f8f-891a-6300d45c728d',
 'mysqlndbcluster-576ae010-46d0-45c9-b19c-2ee170e665fc',
 'zts-event-client-202784c9-b854-4eb8-a517-13ef4fb542b5',
 'nrf-client-nfdiscovery-01bf6da6-90c5-4b01-8ee6-c987962e1ca1',
 'fluent-bit-6319377e-5eeb-4cf7-8fe5-3d7f391cb48e',
 'integration-2927091c-834a-4892-a674-ad47f0ce8dcf',
 'consulagent-container-be9c35a4-85ec-4af5-9169-688c637b7a63',
 'sentinel-5f5789fb-b5a0-44a1-8b93-119fb7488e05',
 'job-orchestrator-service-802ad4ab-e2ae-4ad2-b4e7-d17b5ff45e7e',
 'umalarm-f76ab50

In [12]:
from sklearn.preprocessing import StandardScaler
import random

def tensor_builder(container_name_pod_id):
    
    features = ['container_cpu_utilization','container_memory_utilization','container_memory_working_set']
    scaled_features = ['scaled_container_cpu_utilization','scaled_container_memory_utilization','scaled_container_memory_working_set']

    ##pick random df, and normalize
    container_fe_df = filtered_container_df.loc[(filtered_container_df["container_name_pod_id"] == container_name_pod_id)]
    container_fe_df = container_fe_df.sort_values(by='Timestamp').reset_index(drop=True)
    container_fe_df_len = len(container_fe_df)


    #tensor builder
    start = random.choice(range(container_fe_df_len-25))
    container_fe_df = container_fe_df[start:start+25]

    #scaler transformations
    scaler = StandardScaler()
    container_fe_df[scaled_features] = scaler.fit_transform(container_fe_df[features])

    return container_fe_df

In [13]:
%%time
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
results = multiprocessing.Pool(num_cores).map(tensor_builder, selected_container_list)

96
CPU times: user 5.95 s, sys: 10.8 s, total: 16.7 s
Wall time: 11min 3s


In [14]:
len(results)

10000

In [15]:
results[234].shape

(25, 9)

In [16]:
results[8576].shape

(25, 9)

In [17]:
results[8576]

,Timestamp,container_name_pod_id,container_cpu_utilization,container_memory_utilization,container_memory_working_set,freq,scaled_container_cpu_utilization,scaled_container_memory_utilization,scaled_container_memory_working_set
248,2022-05-05 05:00:53.652,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
249,2022-05-05 05:02:04.088,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
250,2022-05-05 05:02:56.321,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
251,2022-05-05 05:03:52.182,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
252,2022-05-05 05:05:01.233,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
253,2022-05-05 05:07:00.317,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
254,2022-05-05 05:07:47.780,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
255,2022-05-05 05:10:00.214,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
256,2022-05-05 05:10:56.905,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0
257,2022-05-05 05:11:59.169,whereabouts-f671176b-d4f5-4d34-b14a-b8e952199af7,0.0,0.00464,12406784,1156,0.0,0.0,0.0


In [18]:
container_training_df = pd.concat(results)

In [19]:
container_training_df.shape

(250000, 9)

In [20]:
import numpy as np
container_training_tensor = np.array(list(map(lambda x: x.to_numpy(), results)))

In [21]:
container_training_tensor.shape

(10000, 25, 9)

In [22]:
from eks_ml_pipeline import write_tensor, awswrangler_pandas_dataframe_to_s3

In [23]:
write_tensor(container_training_tensor, "dish-5g.core.pd.g.dp.eks.logs.e", "container_pca_ad", "v0.0.1", "training_2022_5_5")

'dish-5g.core.pd.g.dp.eks.logs.e/container_pca_ad/v0.0.1/data/tensors/training_2022_5_5.npy'

In [25]:
awswrangler_pandas_dataframe_to_s3(container_training_df,  "dish-5g.core.pd.g.dp.eks.logs.e", "container_pca_ad", "v0.0.1", "training_2022_5_5")

sucess


In [3]:
import numpy as np
node_tensor = np.zeros((32000,20,3))


In [4]:
node_tensor.shape

(32000, 20, 3)

In [5]:
print(node_tensor)
0.3

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [2]:
from msspackages import get_features

In [3]:
get_features()

,feature_name,feature_description,feature_datatype,feature_type,feature_group_name,feature_group_description,model_type,problem_type,created_by,version,model_parameters
0,container_cpu_utilization,Percentage of utilization of total Node CPU ca...,float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.1,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
1,container_memory_utilization,Percentage of node RAM used by the container,float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.1,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
2,container_memory_working_set,"accessed memory, dirty memory, and kernel memory",float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.1,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
3,container_cpu_utilization,Percentage of utilization of total Node CPU ca...,float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.2,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
4,container_memory_utilization,Percentage of node RAM used by the container,float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.2,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
5,container_memory_working_set,"accessed memory, dirty memory, and kernel memory",float,existing,container_autoencoder_ad,numerical time series data about containers ru...,autoencoder,anomaly detection,"David Cherney, Vinayak Sharma",v0.0.2,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
0,container_cpu_utilization,Percentage of utilization of total Node CPU ca...,float,existing,container_pca_ad,numerical time series data about containers ru...,pca,anomaly detection,David Cherney,v0.0.1,"{'time_steps': 25, 'batch_size': 10, 'train_sa..."
1,container_memory_utilization,Percentage of node RAM used by the container,float,existing,container_pca_ad,numerical time series data about containers ru...,pca,anomaly detection,David Cherney,v0.0.1,"{'time_steps': 25, 'batch_size': 10, 'train_sa..."
2,container_memory_working_set,"accessed memory, dirty memory, and kernel memory",float,existing,container_pca_ad,numerical time series data about containers ru...,pca,anomaly detection,David Cherney,v0.0.1,"{'time_steps': 25, 'batch_size': 10, 'train_sa..."
0,node_cpu_utilization,The percentage of CPU cores utilized on a per ...,float,existing,node_autoencoder_ad,general features to track the infrastructure u...,autoencoder,anomaly detection,Vinayak Sharma,v0.0.1,"{'time_steps': 20, 'batch_size': 36, 'train_sa..."
